In [ ]:
from transformers import AutoModelForCausalLM
from transformers import AutoTokenizer
import torch


# model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270m")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270m-it")

In [ ]:
message = [
    {
        "role":"system",
        "content": "XXX"
    },
    {
        "role":"user",
        "content": "YYY"
    },
    {
        "role":"assistant",
        "content": "ZZZ"
    },
    {
        "role":"user",
        "content": "KKK"
    },
    {
        "role":"assistant",
        "content": "AAA"
    },
    ]

#<bos><start_of_turn>user\nXXX\n\nYYY<end_of_turn>\n<start_of_turn>model\nZZZ<end_of_turn>\n

print(tokenizer.decode(tokenizer.apply_chat_template(message)))


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM


model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270M")

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270M-it")

In [ ]:
messages = [
    {
        'role':"system",
        "content": "You know math well."
    },
    {
        'role':"user",
        "content": "Who are you?"
    },
    # {
    #     'role':"assistant",
    #     "content": "I am Gemma, an open-weights AI assistant. I am a large language model created by the Gemma team.\n"
    # },
    # {
    #     'role':"user",
    #     "content": "And what do you know well? Answer briefly."
    # },
]

with_chat_templ = tokenizer.apply_chat_template(messages, tokenize=False, add_generation_prompt = True)
tokenized = tokenizer.apply_chat_template(messages, tokenize=True, return_tensors="pt", add_generation_prompt = True)

print(tokenized)
with_chat_templ

In [ ]:
tokenized = tokenizer.encode("Einstein was born", return_tensors="pt")
outp = model.generate(tokenized, attention_mask = tokenized.ne(tokenizer.pad_token_id).long(), max_new_tokens=128, temperature=0.000001, top_p = 0.95, output_scores=True, return_dict_in_generate=True)

print(tokenizer.decode(outp[0]))

In [ ]:
import torch
import torch.nn.functional as F

generated_ids = outp.sequences[0]
prompt_len = tokenized.shape[1]

for step, logits in enumerate(outp.scores):
    token_id = generated_ids[prompt_len + step].item()
    token_str = tokenizer.decode([token_id])

    probs = F.softmax(logits[0], dim=-1)
    prob = probs[token_id].item()

    print(
        f"Step {step:03d} | Token: {token_str!r} | "
        f"Prob: {prob:.6f} | Logit: {logits[0, token_id].item():.4f}"
    )


In [ ]:
tokenized.ne(tokenizer.pad_token_id).long()

In [ ]:
# <bos>Einstein was born in 1879 in Ulm, Germany. He was the son of a German physician and a Jewish woman. He was the first to publish a paper on the photoelectric effect. He was the first to publish a paper on the photoelectric effect. He was the first to publish a paper on the photoelectric effect. He was the first to publish a paper on the photoelectric effect. He was the first to publish a paper on the photoelectric effect.

In [ ]:
outp[0]

In [ ]:
tokenizer.decode(236743)
tokenizer.decode(245237)

In [ ]:
len(tokenizer)

In [ ]:
from tqdm import tqdm
import json

dictionary = {}

for i in tqdm(range(len(tokenizer))):
    normalized = tokenizer.decode([i])
    dictionary[i] = normalized

with open("data.json", "w", encoding="utf-8") as f:
    json.dump(dictionary, f, indent=2, ensure_ascii=False)


In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM
from hfdebugger import debug_huggingface_model

model = AutoModelForCausalLM.from_pretrained("google/gemma-3-270M")
tokenizer = AutoTokenizer.from_pretrained("google/gemma-3-270M-it")
debug_huggingface_model(model, tokenizer=tokenizer, input_text="Einstein was born", save_debug_data=True)

Registering hooks on ALL modules... (filtering by name in hook)
Registered 313 hooks (filtering applied in hook function)
Input text: 'Einstein was born'
Tokenized: ['<bos>', 'Einstein', '▁was', '▁born']
Input IDs shape: torch.Size([1, 4])
Input IDs: tensor([[     2, 147505,    691,   8132]])

000_model.embed_tokens: Gemma3TextScaledWordEmbedding (model.embed_tokens)

  → INPUT[0]: model.embed_tokens_input_0
     Shape: [1, 4]
     Dtype: torch.int64
     Mean: 39082.500000, Std: 72375.328125
     First 10: [2, 147505, 691, 8132]
     Last 10:  [2, 147505, 691, 8132]
     Zeros: 0, Total: 4

  → OUTPUT[0]: model.embed_tokens_output
     Shape: [1, 4, 640]
     Dtype: torch.float32
     Mean: -0.010992, Std: 1.031457
     First 10: [-0.36131492257118225, 0.27021417021751404, 0.3381537199020386, 0.3149925172328949, 0.12352646887302399, -0.07913414388895035, 0.10345342010259628, 0.09496097266674042, 0.04844553768634796, -0.007479141931980848]
     Last 10:  [0.8461563587188721, 1.19820678

(<hfdebugger.HFModelDebugger at 0x21348436350>,
 CausalLMOutputWithPast(loss=None, logits=tensor([[[-11.8900,   5.9165,  -0.8929,  ..., -11.9101, -11.9250, -11.8976],
          [ -9.5229,  12.4832,   2.6752,  ...,  -9.5099,  -9.6100,  -9.5799],
          [-16.1099,  12.6004,  -3.7262,  ..., -16.0408, -16.0792, -16.1014],
          [-22.2579,   8.5178,  -4.3568,  ..., -22.3150, -22.3765, -22.3794]]]), past_key_values=DynamicCache(layers=[DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicSlidingWindowLayer, DynamicLayer]), hidden_states=(tensor([[[-0.3613,  0.2702,  0.3382,  ..., -0.1536,  0.0787, -1.7788],
          [-0.5312,  1.0